In [ ]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
from pathlib import Path
from fm_analycts_parametrs import DEF_STATS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# _df = fm_create_dataframe(path='data/italia_novo_2024.html')

p = Path.cwd()/'data'/'raw'/'Divisoes' 
arquivos = [f for f in p.iterdir() if f.is_file()]



# _df = fm_create_dataframe(path='data/SERIEC_2024.html')

In [ ]:
_df_concat = pd.DataFrame()
for arquivo in arquivos:
    print(arquivo)
    _df = fm_create_dataframe(path=str(arquivo))
    
    _df_concat = pd.concat([_df_concat, _df], ignore_index=True)
    

In [ ]:
df = _df_concat.copy()

In [ ]:
sorted(df.columns.to_list())

In [ ]:
with open('ligas.json', "r", encoding="utf-8") as f:
        data = json.load(f)
            
df_ligas = pd.DataFrame(data['ligas'])
display(df_ligas[['nome', 'pais', 'tier']].sort_values(by=['pais','tier'], ignore_index=True))

In [ ]:
df['entrega_cria'] = ((df['cruz_c_p100']*0.30 + df['passe_c_p100']*0.70)/100 )*df['coef']

temp_df = df[['divisao','coef']].groupby(by='divisao', dropna=False).agg(qtd=('coef', 'count'))
temp_df

In [ ]:
df['aof2_p90'] = (
        df['passe_prog_p90'] + 
        df['passe_dec_p90'] + 
        df['grandes_chances_p90'] +
        df['cruz_c_p90'] + 
        df['np_chutes_p90'] + 
        df['fintas_p90']).round(2)

In [ ]:
columns = ['nome','clube','xA_p90','ass_p90','passe_c_p90', 'passe_c_p100','passe_dec_p90',
           'passe_prog_p90', 'grandes_chances_p90','ass_p_aof','gc_p_aof','aof_p90','aof2_p90','aval_cria','entrega_cria']

In [ ]:
df[columns].sort_values(by=['entrega_cria'],ascending=False, ignore_index=True)[:20]

In [ ]:
# df['divisao'].unique().tolist()

sorted(df.columns.to_list())

In [ ]:
df['aval_def_adj'] = df['aval_def']*df['coef']
df['rtg_jg_ar_adj'] = df['rtg_jg_ar']*df['coef']
df['rtg_des_adj'] = df['rtg_des']*df['coef']
df['rtg_adef_adj'] = df['rtg_adef']*df['coef']
df['rtg_duel_adj'] = df['rtg_adef']*df['coef']

p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj1'] = (((     (df['rtg_jg_ar_adj']*df['jg_ar_t_p90']*p1) +
                                        df['rtg_duel_adj']*p2 + 
                                        (df['rtg_des_adj']*df['des_t_p90'])*p3 + 
                                        (df['poss_g_p90']/df['adef_t_p90'])*p4+
                                        df['rtg_adef_adj']*p5
                                        ))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj2'] = ((((df['rtg_jg_ar_adj']*p1) + df['rtg_duel_adj']*p2 + (df['rtg_des_adj'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef_adj']*p5))).round(2)

In [ ]:
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
df['aval_def_adj3'] = ((((df['rtg_jg_ar']*p1) + df['rtg_duel']*p2 + (df['rtg_des'])*p3 + (df['poss_g_p90']/df['adef_t_p90'])*p4 + df['rtg_adef']*p5))).round(2)
df['aval_def_adj4'] = (df['aval_def_adj3']*df['coef']).round(2)

In [ ]:
df_zagueiros = df[df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]
df_zagueiros = df_zagueiros[df_zagueiros['divisao'] == 'Serie C NOW Girone C']
df_zagueiros['aval_def_adj4_pct'] = (df_zagueiros['aval_def_adj4'].rank(pct=True)).round(2)

In [ ]:
df_zagueiros[['nome', 'idade','clube', 'divisao','preco_min', 'posicao_analise', 'aval_def', 'aval_def_adj','aval_def_adj1','aval_def_adj2','aval_def_adj3','aval_def_adj4','aval_def_adj4_pct']].sort_values(by=['aval_def_adj3'],ascending=False, ignore_index=True)

In [ ]:
df_zagueiros['aval_def_adj4'].describe()

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)